In [25]:
import time
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [60]:
url = " https://food.grab.com/sg/en"

chrome_options = webdriver.ChromeOptions()
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134"
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

In [61]:
try:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Accept")]'))).click()
except TimeoutException:
    pass

In [62]:
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".ant-layout")))

location_input = driver.find_element(By.ID, 'location-input')
location_input.click()
time.sleep(2)
location_input.clear()

location_input.send_keys("PT Singapore - Choa Chu Kang North 6, Singapore, 689577")

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".ant-btn.submitBtn___2roqB.ant-btn-primary")))

submit_button = driver.find_element(By.CSS_SELECTOR, '.ant-btn.submitBtn___2roqB.ant-btn-primary')
submit_button.click()

In [63]:
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".ant-layout")))

last_height = driver.execute_script("return document.body.scrollHeight")
t=0
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [64]:
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "RestaurantListRow___1SbZY")))
layout_div = driver.find_element(By.CLASS_NAME, 'RestaurantListRow___1SbZY')

WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, "name___2epcT")))

<selenium.webdriver.remote.webelement.WebElement (session="36e820bce805ad22d964a0207b933e19", element="f.E4C10AA3F938651C2069DA61A1BC9AEB.d.F3AFA23B413A5A215814F24329C95E8F.e.79")>

In [65]:
ratings = layout_div.find_elements(By.CLASS_NAME, 'numbers___2xZGn')

soup = BeautifulSoup(driver.page_source , 'html.parser')

In [66]:
details = soup.find_all('div', class_='ant-row-flex ant-row-flex-start ant-row-flex-top asList___1ZNTr')

In [67]:
import pandas as pd

# Initialize empty lists to store extracted details
restaurant_names = []
restaurant_cuisines = []
ratings = []
direction = []
delivery_times = []
distances = []
discount= []
images = []
promo = []

# Loop through each div element
for div in details:
    # Extract restaurant name
    restaurant_name = div.find('p', class_='name___2epcT').text.strip()
    restaurant_names.append(restaurant_name)
    
    # Extract restaurant cuisine
    cuisine_div = div.find('div', class_='cuisine___T2tCh')
    cuisines = cuisine_div.text.strip() if cuisine_div else None
    
    restaurant_cuisines.append(cuisines)
    
    rating_div = div.find('div', class_='numbersChild___2qKMV')
    if rating_div.find('div', class_='ratingStar'):
        ratings.append(rating_div.text)
    else:
        ratings.append(None)

    directions = div.find_all('div', class_="numbersChild___2qKMV")
    for direction_div in directions:
        if direction_div.find('div', class_="deliveryClock"):
            direction.append(direction_div.text)
    
    discount_div = div.find('div', class_="colInfo___3iLqj").find('div', class_="discount___3h-0m")
    if discount_div: 
        discount.append(discount_div.text)
    else:
        discount.append(None)
    
    image = div.find('img', class_="realImage___2TyNE")
    images.append(image.get('src'))
    
    if div.find('div', class_="promoTagHead___1bjRG"):
        promo.append(True)
    else:
        promo.append(False)
        
    
for x in direction:
        delivery_time, distance  = x.split('•')
        delivery_times.append(delivery_time[:7])
        distances.append(distance[-6:])
        
df = pd.DataFrame({
    'Restaurant Name': restaurant_names,
    'Restaurant Cuisine': restaurant_cuisines,
    'Restaurant Rating': ratings,
    'Restaurant Delivery Time': delivery_times,
    'Restaurant Distance': distances,
    'Discount' : discount,
    'Promo' : promo,
    'Images' : images
})

In [95]:
import pandas as pd

# Read the CSV file into a DataFrame
data = pd.read_csv('data.csv')

# Total count of records
total_count = len(data)

# Null/Non-null statistics
null_non_null_stats = {}
for column in data.columns:
    null_count = data[column].isnull().sum()
    non_null_count = data[column].notnull().sum()
    null_non_null_stats[column] = {'null': null_count, 'non_null': non_null_count}

# Print total count
print(f"Total Count of Records: {total_count}")

# Print null/non-null statistics
print("Null/Non-null Statistics:")
for column, stats in null_non_null_stats.items():
    print(f"Column: {column}, Null: {stats['null']}, Non-null: {stats['non_null']}")

Total Count of Records: 277
Null/Non-null Statistics:
Column: Unnamed: 0, Null: 0, Non-null: 277
Column: Restaurant Name, Null: 0, Non-null: 277
Column: Restaurant Cuisine, Null: 0, Non-null: 277
Column: Restaurant Rating, Null: 42, Non-null: 235
Column: Restaurant Delivery Time, Null: 0, Non-null: 277
Column: Restaurant Distance, Null: 0, Non-null: 277
Column: Discount, Null: 248, Non-null: 29
Column: Promo, Null: 0, Non-null: 277
Column: Images, Null: 0, Non-null: 277
